In [2]:
pip install emoji


  Obtaining dependency information for emoji from https://files.pythonhosted.org/packages/bd/48/508a980e9c9f12dbc2e45e57f027abdd92320e5be37c5c551588a86acb48/emoji-2.11.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/433.8 kB ? eta -:--:--
   --------------------------------------  430.1/433.8 kB 13.5 MB/s eta 0:00:01
   --------------------------------------- 433.8/433.8 kB 13.7 MB/s eta 0:00:00


In [41]:
import re
import csv
import emoji
import json
import requests
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [42]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhavp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhavp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhavp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [50]:
def get_top_posts(subreddit, limit=250):
    url = f"https://api.pullpush.io/reddit/search/submission/?subreddit={subreddit}&sort=desc&sort_type=score&limit={limit}"
    response = requests.get(url)
    return response.json()['data']

In [51]:

def get_comments(post_id):
    url = f"https://api.pullpush.io/reddit/comment/search/?link_id={post_id}"
    response = requests.get(url)
    return response.json()['data']

In [52]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string


In [53]:
def clean_comment(comment):
    if comment in ['[deleted]', '[removed]']:
        return None

    #lowercase
    comment = comment.lower()

    #user mentions
    comment = re.sub(r'@\w+', '', comment)

    #emojis
    comment = emoji.demojize(comment)

    #useless chars
    comment = ''.join(ch for ch in comment if ch.isalnum() or ch.isspace())

    #tokenize
    words = word_tokenize(comment)

    #stopwords
    words = [word for word in words if word not in stopwords.words('english')]

    #short words
    words = [word for word in words if len(word) > 2]

    #lemma
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

top_posts = get_top_posts('politics')


In [54]:
with open('data.json', 'w', encoding='utf-8') as file:
    file.write('[')
    for i, post in enumerate(top_posts):
        comments = get_comments(post['id'])
        cleaned_comments = [clean_comment(comment['body']) for comment in comments[:100]]
        cleaned_comments = [comment for comment in cleaned_comments if comment is not None]
        data = {
            "Title": post['title'],
            "Comments": cleaned_comments
        }
        json.dump(data, file, ensure_ascii=False)
        if i < len(top_posts) - 1:  #skip last comma, formatting nonsense
            file.write(',')
    file.write(']')

In [55]:
with open('data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Now you can work with the data as needed
total_comments = sum(len(post["Comments"]) for post in data)

print("Total number of comments:", total_comments)

Total number of comments: 8034


In [56]:
print(data)

[{'Title': 'Megathread: Joe Biden Projected to Defeat President Donald Trump and Win the 2020 US Presidential Election', 'Comments': ['httpsanchorfmnightbanterepisodess2ep4whyiwouldntwantawomanpresidentev8tsq', 'there hundred thousand dead muslim would like disagree sir', 'gttrump fucking rest world sorry missed point president elected november 3rd american pay tax american government protect interest american citizen sure president global influence end there reason potus although fair think agree itd lot fun hear secret service refer president using word pee gti want stop hearing name everyday news turn watch local news bigger news network arent going give trump simply brings viewer take look abandoned ruin cnn nowadays 247 trump twitter reporting tried move away theyve lost entire audience news network unlikely make mistake gtbiden seems formal world need excessive formality lack action biden obama clinton got trump elected first place gtis bad american image argue constant drone str